In [6]:
import os
from os.path import join

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from o2_utils.selectors import find_files_from_pattern
from tqdm.notebook import tqdm
import videochef as vc

In [2]:
base_path = '/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/'
save_loc = join(base_path, '../kp_detection/scale_annotation/images')
n_frames = 30 * 60 * 120
videos = find_files_from_pattern(base_path, 'J043*/20240201_J043*/*.mp4', error_behav="pass", exclude_patterns=["azure"])
videos[:5]

['/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.top.mp4',
 '/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.side1.mp4',
 '/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.side3.mp4',
 '/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.side2.mp4',
 '/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.side4.mp4']

In [3]:
videos = [videos[0]]

In [15]:
for vid in videos:

    print(vid)

    recording_name = os.path.basename(vid).split('.')[0]  # ie 20240201_J04301
    save_dir = os.path.join(save_loc, recording_name)
    os.makedirs(save_dir, exist_ok=True)

    # Prep the hist eq
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32, 32))

    # Choose which frames to annotate
    frames = np.sort(np.random.choice(n_frames, 150, replace=False))
    use_frames = frames[:75] if 'side' in vid else frames

    # Prep the video reader
    reader = imageio.get_reader(vid)
    im_prefix = os.path.basename(vid).split('.mp4')[0]

    # Save the frames
    print("Saving frames...")
    # for i in tqdm(use_frames):
    #     im_path = f'{im_prefix}.{i}.jpg'
    #     im = reader.get_data(i)[:,:,0]
    #     im = clahe.apply(im)
    #     im = np.repeat(im[:,:,None],3,axis=2)
    #     cv2.imwrite(im_path, im)

    # Save the frames
    with vc.io.VideoReader(vid, frame_ixs=use_frames) as reader:
        for i, im in enumerate(tqdm(reader, total=len(use_frames))):
            im_path = f'{im_prefix}.{use_frames[i]}.jpg'
            im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            im_gray_8bit = cv2.convertScaleAbs(im_gray)
            im = clahe.apply(im_gray_8bit)
            im = np.repeat(im[:, :, None], 3, axis=2)
            cv2.imwrite(join(save_dir, im_path), im)



/n/groups/datta/Jonah/20240125_CoA_hM4Di_6cam/data/J04302/20240201_J04302/20240201_J04302.top.mp4
Saving frames...


  0%|          | 0/150 [00:00<?, ?it/s]

In [16]:
os.listdir(save_loc)

['20240201_J04302']